In [1]:
import os
import toml
import logging
import numpy as np
import torch
import cv2
import pandas as pd
from PIL import Image
from meegnet_functions import load_single_subject
import matplotlib.pyplot as plt
from meegnet.parsing import parser, save_config
from meegnet.network import Model
from meegnet.viz import plot_masked_epoch
import cv2
from pytorch_grad_cam import GradCAM

LOG = logging.getLogger("meegnet")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

/home/arthur/.pyvenv/meegnet/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
###############
### PARSING ###
###############

# For Jupyter Notebook Only: we create fake args
argstring = "--config ../scripts/eventclf.ini".split()
args = parser.parse_args(argstring)
save_config(vars(args), args.config)
with open("../default_values.toml", "r") as f:
    default_values = toml.load(f)

fold = None if args.fold == -1 else int(args.fold)
if args.clf_type == "eventclf":
    assert (
        args.datatype != "rest"
    ), "datatype must be set to passive in order to run event classification"

if args.feature == "bins":
    trial_length = default_values["TRIAL_LENGTH_BINS"]
elif args.feature == "bands":
    trial_length = default_values["TRIAL_LENGTH_BANDS"]
elif args.feature == "temporal":
    trial_length = default_values["TRIAL_LENGTH_TIME"]

if args.clf_type == "subclf":
    trial_length = int(args.segment_length * args.sfreq)

if args.sensors == "MAG":
    n_channels = default_values["N_CHANNELS_MAG"]
elif args.sensors == "GRAD":
    n_channels = default_values["N_CHANNELS_GRAD"]
else:
    n_channels = default_values["N_CHANNELS_OTHER"]

input_size = (
    (1, n_channels, trial_length)
    if args.flat
    else (
        n_channels // default_values["N_CHANNELS_MAG"],
        default_values["N_CHANNELS_MAG"],
        trial_length,
    )
)

name = f"{args.clf_type}_{args.model_name}_{args.seed}_{args.sensors}"
suffixes = ""
if args.net_option == "custom_net":
    if args.batchnorm:
        suffixes += "_BN"
    if args.maxpool != 0:
        suffixes += f"_maxpool{args.maxpool}"

    name += f"_dropout{args.dropout}_filter{args.filters}_nchan{args.nchan}_lin{args.linear}_depth{args.hlayers}"
    name += suffixes

n_samples = None if int(args.n_samples) == -1 else int(args.n_samples)
if args.clf_type == "subclf":
    data_path = os.path.join(args.save_path, f"downsampled_{args.sfreq}")
    n_subjects = len(os.listdir(data_path))
    n_outputs = min(n_subjects, args.max_subj)
    lso = False
else:
    n_outputs = 2
    lso = True

In [3]:
######################
### LOGGING CONFIG ###
######################

if args.log:
    log_name = f"{args.model_name}_{args.seed}_{args.sensors}"
    if fold is not None:
        log_name += f"_fold{args.fold}"
    log_name += "_gradcam_computations.log"
    log_file = os.path.join(args.save_path, log_name)
    logging.basicConfig(filename=log_file, filemode="a")
    LOG.info(f"Starting logging in {log_file}")

07/16/2024 09:44:54 PM Starting logging in /home/arthur/data/meegnet_42_ALL_gradcam_computations.log


In [4]:
##############################
### PREPARING SAVE FOLDERS ###
##############################

viz_path = os.path.join(args.save_path, "visualizations")
if not os.path.exists(viz_path):
    os.makedirs(viz_path)

In [5]:
#####################
### LOADING MODEL ###
#####################

if args.model_path is None:
    model_path = args.save_path
else:
    model_path = args.model_path

if not os.path.exists(model_path):
    logging.info(f"{model_path} does not exist. Creating folders")
    os.makedirs(model_path)

my_model = Model(name, args.net_option, input_size, n_outputs, save_path=args.save_path)
my_model.from_pretrained()
net = my_model.net
# my_model.load()

07/16/2024 09:44:55 PM => loading checkpoint '/home/arthur/.cache/huggingface/hub/models--lamaroufle--meegnet/snapshots/5f96fe8d1b9ce85462329cdb3f148e83d3383873/eventclf_meegnet_3_102_400_2.pt'


In [6]:
####################
### LOADING DATA ###
####################

csv_file = os.path.join(args.save_path, f"participants_info.csv")
dataframe = (
    pd.read_csv(csv_file, index_col=0)
    .sample(frac=1, random_state=args.seed)
    .reset_index(drop=True)[: args.max_subj]
)
subj_list = dataframe["sub"]
n_sub = len(subj_list)

In [ ]:
#################
### MAIN LOOP ###
#################

for i, layer in enumerate(net.feature_extraction):
    target_layers = [layer]
    gradcam = GradCAM(model=net, target_layers=target_layers)

    all_cams = []
    all_trials = None
    for sub in subj_list:
        data = load_single_subject(sub, n_samples, lso, args).data
        if data == []:
            continue
        input_tensor = data.to(torch.float)  # can be multiple images

        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cam = gradcam(input_tensor=input_tensor, targets=None)

        # In this example grayscale_cam has only one image in the batch:
        all_cams.append(np.mean(grayscale_cam, 0))
        if all_trials is None:
            all_trials = data.mean(axis=0)[0] / n_sub
        else:
            all_trials += data.mean(axis=0)[0] / n_sub

    all_cams = np.array(all_cams)
    cams = np.mean(all_cams, axis=0)
    cams_img = cv2.merge([cams, cams, cams])

    img = Image.fromarray(np.uint8(255 * cams_img))
    img.save(os.path.join(viz_path, f"gradcam_masklayer{i}.png"))

    plot_masked_epoch(all_trials, cams)
    plt.savefig(os.path.join(viz_path, f"average_trial_layer{i}.png"), dpi=400)

07/16/2024 09:44:55 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:55 PM Loading subject CC410220
07/16/2024 09:44:56 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:56 PM Loading subject CC221324
07/16/2024 09:44:56 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:56 PM Loading subject CC310252
07/16/2024 09:44:57 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:57 PM Loading subject CC420167
07/16/2024 09:44:57 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:57 PM Loading subject CC722891
07/16/2024 09:44:57 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:57 PM Loading subject CC620429
07/16/2024 09:44:58 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:58 PM Loading subject CC510648
07/16/2024 09:44:58 PM Logging subjects and labels from /home/arthur/data...
07/16/2024 09:44:58 PM Loading subject CC420322
